<a href="https://colab.research.google.com/github/DanielDialektico/rag_agentes_langchain_curso/blob/main/notebooks/langchain_agent_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://dialektico.com/wp-content/uploads/2023/03/MiniLogoW4.png" alt="Dialéktico Logo" />

Este pequeño tutorial pertenece al curso de RAG y agentes con LangChain al que puedes acceder mediante la siguiente URL: https://www.youtube.com/playlist?list=PLlWTv9_GeWd32stuEMWpYOnxiVxnXaU6q

Sigue los videos del curso para recibir instrucciones y contexto sobre la ejecución de este Notebook.

<br>

# Se instalan e importan las librerías

In [ ]:
!pip install langchain==0.3.20
!pip install langchain_deepseek==0.1.2
!pip install langgraph==0.3.31

In [ ]:
import os
from langchain_deepseek import ChatDeepSeek
from langchain_core.messages import  HumanMessage, AIMessage
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from google.colab import userdata
import warnings

warnings.filterwarnings("ignore")

## Se añade valor de API key mediante un secreto

In [ ]:
# Se añade la API key como variable de ambiente desde un secreto en Colab.
os.environ["DEEPSEEK_API_KEY"] = userdata.get('DEEPSEEK_API_KEY')

## Se declara el modelo a utilizar

In [ ]:
# Se define el modelo y añaden valores de parámetros.
model = ChatDeepSeek(
      model="deepseek-chat",
      temperature=0,
      max_tokens=100
      )

<br>

# Herramientas de apoyo para los modelos

​En LangChain, una herramienta (tool) es una función de Python asociada a un esquema que define su nombre, descripción y los argumentos esperados. Estas herramientas permiten que modelos de lenguaje, cadenas o agentes interactúen con el mundo exterior, ejecutando acciones específicas o accediendo a información externa.

## Creando una herramienta con LangChain

In [ ]:
#Se crea una función para ser utilizada como herramienta.

def dialektinumero(number_1: int, number_2: int) -> float:
    """Crea un dialektinúmero mediante una operación específica
    con dos números.

    Args:
        number_1: primer número de entrada.
        number_2: segundo número de entrada.

    Returns:
        El resultado de la operación.
    """
    result = (number_1 + number_2 - 1.5)*3.14
    return result

In [ ]:
dialektinumero(2,8)

<br>

## LLM utilizando la herramienta (agente)

Se pasa una lista de herramientas a la LLM mediante `.bind_tools`.

In [ ]:
# Se añade la función a una lista de herramientas.
tools = [dialektinumero]

# Se añade la función como herramienta del LLM.
model_with_tools = model.bind_tools(tools)

# Se imprime la configuración.
model_with_tools

Ejecutamos un mensaje que no requiera el uso de la herramienta.

In [ ]:
# Se genera una respuesta y se imprime su contenido.
response = model_with_tools.invoke([HumanMessage(content="Hola!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

In [ ]:
# Se obtienen los detalles.
response

Probamos con un prompt que requiera el uso de la herramienta.

In [ ]:
response = model_with_tools.invoke([HumanMessage(content="¿Cuáles es el dialektinúmero de 5 y 10?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

<br>

## Utilizando al agente

Ahora que se ha definido una herramienta y el modelo de lenguaje (LLM), podemos crear un agente. Usaremos LangGraph para construir el agente.

Ahora podemos inicializar el agente con el LLM y las herramientas.

**Nota:** que estamos pasando el modelo, no model_with_tools. Esto se debe a que create_react_agent llamará a .bind_tools por nosotros internamente.

In [ ]:
# Se crea el agente, pasando el modelo y las herramientas definidas.
agent_executor = create_react_agent(model, tools)

In [ ]:
# Se pasa un prompt de entrada al agente.
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="¿Cuáles es el dialektinúmero de 5 y 10?")]}
)
response["messages"]

In [ ]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="¿Cuáles es el dialektinúmero de 5 y 10?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
# Filtrar el mensaje de tipo AIMessage (aunque usualmente es el último)
ai_messages = [m for m in response["messages"] if isinstance(m, AIMessage)]
print(ai_messages[-1].content)

<br>

Herramientas disponibles por LangChain: https://python.langchain.com/docs/integrations/tools/

Más información en: https://python.langchain.com/docs/concepts/tools/

<br>

In [ ]:
# Dialektico Machine learning practices © 2025 by Daniel Antonio García Escobar
# is licensed under CC BY-NC 4.0. To view a copy of this license,
# visit https://creativecommons.org/licenses/by-nc/4.0/

# Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International
# Public License